In [ ]:
import pandas as pd
import glob
from google.colab import files
from datetime import datetime

In [ ]:
uploaded = files.upload()

Saving Mansarovar2024.csv to Mansarovar2024.csv
Saving Mansarovar2023.csv to Mansarovar2023.csv


In [ ]:
# 2. Get all CSV file paths
file_paths = glob.glob("/content/*.csv")
print(f"Found {len(file_paths)} CSV files to merge")

Found 2 CSV files to merge


In [ ]:
# 3. Function to standardize timestamp format
def standardize_timestamp(ts):
    try:
        # Try parsing with dayfirst=True (DD-MM-YYYY)
        dt = pd.to_datetime(ts, dayfirst=True)
        return dt.strftime('%d-%m-%Y %H:%M')
    except:
        try:
            # Try parsing ISO format (YYYY-MM-DD)
            dt = pd.to_datetime(ts)
            return dt.strftime('%d-%m-%Y %H:%M')
        except:
            # Try parsing other common formats
            for fmt in ['%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M', '%d-%m-%Y %H:%M:%S']:
                try:
                    dt = datetime.strptime(ts, fmt)
                    return dt.strftime('%d-%m-%Y %H:%M')
                except:
                    continue
            return None


In [ ]:
# 4. Read and combine files with standardized timestamps
df_list = []
for file in file_paths:
    try:
        # Read CSV without automatic date parsing
        temp_df = pd.read_csv(file)

        # Standardize timestamp format
        temp_df['Timestamp'] = temp_df['Timestamp'].apply(standardize_timestamp)

        # Convert to datetime object for proper sorting
        temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'], format='%d-%m-%Y %H:%M')

        # Drop rows with invalid timestamps
        temp_df = temp_df.dropna(subset=['Timestamp'])

        df_list.append(temp_df)
        print(f"Successfully processed {file} with {len(temp_df)} rows")
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

<ipython-input-4-c2157148167d>:5: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt = pd.to_datetime(ts, dayfirst=True)


Successfully processed /content/Mansarovar2023.csv with 8760 rows


<ipython-input-4-c2157148167d>:5: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt = pd.to_datetime(ts, dayfirst=True)


Successfully processed /content/Mansarovar2024.csv with 8784 rows


In [ ]:
# Add this before merging to ensure all files have same columns
common_columns = set(df_list[0].columns)
for df in df_list[1:]:
    common_columns.intersection_update(df.columns)
print("Common columns across all files:", common_columns)

Common columns across all files: {'NO2 (µg/m³)', 'RH (%)', 'BP (mmHg)', 'AT (°C)', 'NH3 (µg/m³)', 'SR (W/mt2)', 'SO2 (µg/m³)', 'PM2.5 (µg/m³)', 'Season', 'Timestamp', 'WD (deg)', 'CO (mg/m³)', 'PM10 (µg/m³)', 'WS (m/s)', 'Month', 'Ozone (µg/m³)'}


In [ ]:
# 4. Combine all dataframes
merged_df = pd.concat(df_list, ignore_index=True)

In [ ]:
# 5. Sort by timestamp and clean up
merged_df = merged_df.sort_values(by="Timestamp").reset_index(drop=True)

In [ ]:
# 6. Verify the combined data
print("\nCombined Data Summary:")
print(f"Total rows: {len(merged_df)}")
print(f"Date range: {merged_df['Timestamp'].min()} to {merged_df['Timestamp'].max()}")
print("\nMissing values per column:")
print(merged_df.isnull().sum())


Combined Data Summary:
Total rows: 17544
Date range: 2023-01-01 00:00:00 to 2024-12-31 23:00:00

Missing values per column:
Timestamp            0
PM2.5 (µg/m³)        0
PM10 (µg/m³)         0
NO2 (µg/m³)          0
NH3 (µg/m³)          0
SO2 (µg/m³)          0
CO (mg/m³)           0
Ozone (µg/m³)        0
AT (°C)              0
RH (%)               0
WS (m/s)             0
WD (deg)             0
SR (W/mt2)           0
BP (mmHg)        17544
Month                0
Season               0
dtype: int64


In [ ]:
# 7. Save the merged file
output_file = "merged_all_yearsMansarovar.csv"
merged_df.to_csv(output_file, index=False)
print(f"\nSaved merged data to {output_file}")


Saved merged data to merged_all_yearsMansarovar.csv


In [ ]:
# 8. Download the file
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>